In [2]:
!pip install tavily-python
!pip install -U langgraph langchain-community langchain-groq langchain-tavily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requ

In [ ]:


from groq import Groq
from tavily import TavilyClient
import os
from datetime import datetime

# ------------------- CONFIG -------------------
os.environ["GROQ_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = "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"

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

conversation_log = []  # stores all interactions


# ------------------- INTENT DETECTION -------------------
def infer_intent(query):
    prompt = f"""
Classify the main intent of this user query into a short label:
Examples: travel planning, product research, education, finance, health, technology, general knowledge, etc.

Query: "{query}"
Return only the label.
"""
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    return response.choices[0].message.content.strip()


# ------------------- CLARIFIER -------------------
def generate_dynamic_clarifier(query, context, asked_questions, intent):
    prompt = f"""
You are a Clarifier Agent refining research queries.

User query: "{query}"
Detected intent: {intent}
Current context: {context}
Already asked: {asked_questions}

Ask ONE new clarifying question that helps make the query more specific or useful for deep research.
Avoid repetition.
If the query already has enough detail, respond with exactly: 'No further clarification needed.'
"""
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.6,
    )
    return response.choices[0].message.content.strip()


def clarification_phase(user_query):
    intent = infer_intent(user_query)
    print(f"\n[System] 🧩 Inferred intent: {intent}\n")

    context = {}
    asked_questions = []
    rounds = 0
    MAX_QUESTIONS = 3

    while rounds < MAX_QUESTIONS:
        clarifying_question = generate_dynamic_clarifier(user_query, context, asked_questions, intent)
        if "No further clarification needed" in clarifying_question:
            print("\n[Clarifier] ✅ Enough clarity — proceeding with research...\n")
            break

        print(f"[Clarifier] {clarifying_question}")
        user_answer = input("Your answer: ")

        asked_questions.append(clarifying_question)
        context[f"answer_{rounds + 1}"] = {"question": clarifying_question, "answer": user_answer}
        rounds += 1

    return context, intent


# ------------------- RESEARCH -------------------
def summarize_context(context):
    return "; ".join(f"{v['question'].replace('?', '')}: {v['answer']}" for v in context.values())[:400]


def conduct_research(query, context):
    print("\n[Researcher] 🌐 Conducting deep web research...\n")
    summary = summarize_context(context)
    full_query = f"{query}. Context: {summary}"[:400]

    try:
        results = tavily.search(full_query, search_depth="advanced", max_results=6)
        combined_data = "\n\n".join([r.get("content", str(r)) for r in results.get("results", [])])
    except Exception as e:
        combined_data = f"Error retrieving data: {e}"

    return combined_data


# ------------------- REFLECTION -------------------
def reflective_analysis(query, data, context, intent):
    context_summary = summarize_context(context)
    prompt = f"""
You are Deep Researcher — an intelligent agent that interprets live web data for decision making.

User Query: {query}
Intent: {intent}
Clarified Context: {context_summary}

Web Data Retrieved:
{data}

Now write a research report that:
1. Presents direct, factual answers to the query using the available web data.
2. Automatically adapts structure to the query domain.
3. Then include a Reflection Section:
   - Reliability and relevance of results
   - Missing perspectives or gaps
   - 2–3 improvements or next steps
"""
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.6,
    )
    return response.choices[0].message.content.strip()


# ------------------- MEMORY HANDLER (UPDATED) -------------------
def handle_memory_queries(user_query):
    """Handles 'what was my last question' or 'show history' type queries."""
    q = user_query.lower().strip()
    memory_triggers = [
        "last question", "previous question", "what was my last",
        "what did i ask", "show history", "list history", "my history"
    ]

    if any(trigger in q for trigger in memory_triggers):
        if len(conversation_log) == 0:
            return "You haven't asked any research questions yet."

        if "last" in q or "previous" in q:
            last = conversation_log[-1]
            return (f"🧠 Your last question (at {last['timestamp']}) was:\n"
                    f"👉 {last['query']}\n\nSummary:\n{last['summary'][:700]}")

        if "history" in q:
            history = "\n".join(
                [f"{i+1}. {entry['timestamp']} — {entry['query']}" for i, entry in enumerate(conversation_log)]
            )
            return f"🧾 Conversation history:\n{history}"

    return None


# ------------------- MAIN LOOP -------------------
def deep_researcher():
    print("🤖 Welcome to Deep Researcher — powered by Groq + Tavily")
    print("💡 Type 'exit' anytime to end the session.\n")

    while True:
        user_query = input("Please describe what you want to research: ").strip()

        if user_query.lower() in ["exit", "quit", "bye"]:
            print("\n🧠 Session ended. Goodbye!")
            break

        # ✅ MEMORY CHECK COMES FIRST NOW
        memory_reply = handle_memory_queries(user_query)
        if memory_reply:
            print("\n" + memory_reply + "\n")
            continue

        # Clarify → Research → Reflect
        context, intent = clarification_phase(user_query)
        data = conduct_research(user_query, context)
        report = reflective_analysis(user_query, data, context, intent)

        print("\n" + "=" * 80)
        print("\n🧠 FINAL RESEARCH REPORT\n")
        print(report)
        print("\n" + "=" * 80)

        # Save to memory
        conversation_log.append({
            "query": user_query,
            "intent": intent,
            "context": context,
            "summary": report,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })


# ------------------- RUN -------------------
if __name__ == "__main__":
    deep_researcher()


🤖 Welcome to Deep Researcher — powered by Groq + Tavily
💡 Type 'exit' anytime to end the session.

